In [56]:
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torch.nn import Module
from torch.nn import Conv2d
from torch.nn import Linear
from torch.nn import MaxPool2d
from torch.nn import ReLU
from torch.nn import LogSoftmax
from torch import flatten
from sklearn.metrics import classification_report
from torch.utils.data import random_split
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from torchvision.datasets import KMNIST
from torch.optim import Adam
from torch import nn
import matplotlib.pyplot as plt
import numpy as np
import argparse
import torch
import time

from sklearn.preprocessing import MinMaxScaler    
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

In [83]:
class LeNet(Module):
    def __init__(self, numChannels, classes):
        # call the parent constructor
        super(LeNet, self).__init__()
        # initialize first set of CONV => RELU => POOL layers
        self.conv1 = Conv2d(in_channels=numChannels, out_channels=20,
        kernel_size=(5, 5))
        self.relu1 = ReLU()
        self.maxpool1 = MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        # initialize second set of CONV => RELU => POOL layers
        self.conv2 = Conv2d(in_channels=20, out_channels=50,
        kernel_size=(5, 5))
        self.relu2 = ReLU()
        self.maxpool2 = MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
        # initialize first (and only) set of FC => RELU layers
        self.fc1 = Linear(in_features=14450, out_features=500)
        self.relu3 = ReLU()
        # initialize our softmax classifier
        self.fc2 = Linear(in_features=500, out_features=classes)
        self.logSoftmax = LogSoftmax(dim=1)
        
    def forward(self, x):
        # pass the input through our first set of CONV => RELU =>
        # POOL layers
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.maxpool1(x)
        # pass the output from the previous layer through the second
        # set of CONV => RELU => POOL layers
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.maxpool2(x)
        # flatten the output from the previous layer and pass it
        # through our only set of FC => RELU layers
        x = flatten(x, 1)
        x = self.fc1(x)
        x = self.relu3(x)
        # pass the output to our softmax classifier to get our output
        # predictions
        x = self.fc2(x)
        output = self.logSoftmax(x)
        # return the output predictions
        return output

In [84]:
with open("dataset.pickle", "rb") as file:
    dataset = pickle.load(file)
    
X_data = np.array(dataset[0][:])
Y_data = np.array(dataset[1][:])
Y_data_c = np.zeros((len(Y_data), num_classes))
for i, y in enumerate(Y_data):
    Y_data_c[i][y-1] = 1

In [85]:
# define training hyperparameters
INIT_LR = 1e-3
EPOCHS = 10
# set the device we will be using to train the model
device = torch.device("cpu")

X_train, X_test, Y_train, Y_test = train_test_split(X_data, Y_data_c, test_size=0.2, random_state=42)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.25, random_state=21)

# initialize the LeNet model
print("[INFO] initializing the LeNet model...")
device = "cpu"
model = LeNet(numChannels=2, classes=num_classes).to(device)
# initialize our optimizer and loss function
opt = Adam(model.parameters(), lr=INIT_LR)
lossFn = nn.NLLLoss()
# initialize a dictionary to store training history
H = {"train_loss": [],"train_acc": [],"val_loss": [],"val_acc": []}
# measure how long training is going to take
print("[INFO] training the network...")


[INFO] initializing the LeNet model...
[INFO] training the network...


In [101]:
startTime = time.time()
# loop over our epochs
for e in range(0, EPOCHS):
    p = np.random.permutation(len(Y_train))
    X, Y = X_train[p], Y_train[p]
    # set the model in training mode
    model.train()
    # initialize the total training and validation loss
    totalTrainLoss = 0
    totalValLoss = 0
    # initialize the number of correct predictions in the training
    # and validation step
    trainCorrect = 0
    valCorrect = 0
    batch_size = 64
    N = len(Y) // batch_size
    # loop over the training set
    for i in range(N):
        # send the input to the device
        X_batch = torch.from_numpy(X[i*batch_size:(i+1)*batch_size][:])
        Y_batch = torch.from_numpy(Y[i*batch_size:(i+1)*batch_size][:])
        X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)
        # perform a forward pass and calculate the training loss
        pred = model(X_batch.float())
        loss = lossFn(pred, Y_batch.argmax(1))
        # zero out the gradients, perform the backpropagation step,
        # and update the weights
        opt.zero_grad()
        loss.backward()
        opt.step()
        # add the loss to the total training loss so far and
        # calculate the number of correct predictions
        totalTrainLoss += loss
        trainCorrect += (pred.argmax(1) == Y_batch.argmax(1)).type(torch.float).sum().item()
        
#     # switch off autograd for evaluation
    p = np.random.permutation(len(Y_val))
    X_v, Y_v = X_val[p], Y_val[p]
    with torch.no_grad():
    # set the model in evaluation mode
        model.eval()
        # loop over the validation set
        # send the input to the device
        Xv_batch = torch.from_numpy(X_v)
        Yv_batch = torch.from_numpy(Y_v)
        # send the input to the device
        (Xv, Yv) = (Xv_batch.to(device), Yv_batch.to(device))
        # make the predictions and calculate the validation loss
        pred = model(Xv.float())
        totalValLoss = lossFn(pred, Yv.argmax(1))
        # calculate the number of correct predictions
        valCorrect = (pred.argmax(1) == Yv.argmax(1)).type(torch.float).sum().item()

    # calculate the average training and validation loss
    avgTrainLoss = totalTrainLoss / N
    # calculate the training and validation accuracy
    trainCorrect = trainCorrect / len(Y_train)
    valCorrect = valCorrect / len(Y_val)
            
    # update our training history
    H["train_loss"].append(avgTrainLoss.cpu().detach().numpy())
    H["train_acc"].append(trainCorrect)
    H["val_loss"].append(avgValLoss.cpu().detach().numpy())
    H["val_acc"].append(valCorrect)
    # print the model training and validation information
    print("[INFO] EPOCH: {}/{}".format(e + 1, EPOCHS))
    print("Train loss: {:.6f}, Train accuracy: {:.4f}".format(avgTrainLoss, trainCorrect))
    print("Val loss: {:.6f}, Val accuracy: {:.4f}\n".format(totalValLoss, valCorrect))
    # finish measuring how long training took
    
endTime = time.time()
print("[INFO] total time taken to train the model: {:.2f}s".format(endTime - startTime))

[INFO] EPOCH: 1/10
Train loss: 0.000035, Train accuracy: 0.9644
Val loss: 0.086901, Val accuracy: 0.9754

[INFO] EPOCH: 2/10
Train loss: 0.000032, Train accuracy: 0.9644
Val loss: 0.087498, Val accuracy: 0.9754

[INFO] EPOCH: 3/10
Train loss: 0.000031, Train accuracy: 0.9644
Val loss: 0.087798, Val accuracy: 0.9754

[INFO] EPOCH: 4/10
Train loss: 0.000029, Train accuracy: 0.9644
Val loss: 0.088605, Val accuracy: 0.9754

[INFO] EPOCH: 5/10
Train loss: 0.000024, Train accuracy: 0.9644
Val loss: 0.089601, Val accuracy: 0.9754

[INFO] EPOCH: 6/10
Train loss: 0.000026, Train accuracy: 0.9644
Val loss: 0.090029, Val accuracy: 0.9754

[INFO] EPOCH: 7/10
Train loss: 0.000025, Train accuracy: 0.9644
Val loss: 0.089982, Val accuracy: 0.9754

[INFO] EPOCH: 8/10
Train loss: 0.000023, Train accuracy: 0.9644
Val loss: 0.090154, Val accuracy: 0.9754

[INFO] EPOCH: 9/10
Train loss: 0.000022, Train accuracy: 0.9644
Val loss: 0.090920, Val accuracy: 0.9754

[INFO] EPOCH: 10/10
Train loss: 0.000021, Trai

In [102]:

# we can now evaluate the network on the test set
print("[INFO] evaluating network...")
# turn off autograd for testing evaluation
N_t = len(X_test) // batch_size
with torch.no_grad():
# set the model in evaluation mode
    model.eval()
    # loop over the validation set
    # send the input to the device
    Xt_batch = torch.from_numpy(X_test)
    Yt_batch = torch.from_numpy(Y_test)
    # send the input to the device
    (Xt, Yt) = (Xt_batch.to(device), Yt_batch.to(device))
    # make the predictions and calculate the validation loss
    pred = model(Xt.float())
    totalValLoss = lossFn(pred, Yt.argmax(1))
    # calculate the number of correct predictions
    valCorrect = (pred.argmax(1) == Yt.argmax(1)).type(torch.float).sum().item()
    valCorrect = valCorrect / len(Y_test)
    print("Test loss: {:.6f}, Test accuracy: {:.4f}\n".format(totalValLoss, valCorrect))

[INFO] evaluating network...
Test loss: 0.082651, Test accuracy: 0.9836



In [103]:
torch.save(model.state_dict(), "classifier.pt")

### Model Load

In [104]:
device = "cpu"
new_model = LeNet(numChannels=2, classes=num_classes).to(device)
new_model.load_state_dict(torch.load("classifier.pt"))

<All keys matched successfully>